# Slicing SST1 RSoXS Data

## Pip install and restart kernel 

## Imports

In [6]:
## The autoreload IPython magic command reloads all modules before code is ran
%load_ext autoreload

In [9]:
###!pip install -e /nsls2/users/alevin/repos/pyhyper_toneygroup_fork/PyHyperScattering  # to use pip to install via directory
### !pip install zarr  # fixed an error with xr.DataSet.to_zarr() method, though not sure if this is really needed

In [10]:
## Imports
import PyHyperScattering as phs
import pathlib
# import pickle
import sys
# import json
# import zarr
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
# from matplotlib_inline.backend_inline import set_matplotlib_formats

sys.path.append('/nsls2/users/alevin/local_lib')
from andrew_rsoxs_fxns import *

## Some setup functions
# set_matplotlib_formats('svg')
# c = from_profile('rsoxs')
print(f'Using PyHyperScattering Version: {phs.__version__}')
# rsoxsload = phs.load.SST1RSoXSDB(corr_mode='None', use_chunked_loading=True)  # initialize rsoxs databroker loader w/ Dask

## Define masks directory path
userPath = pathlib.Path('/nsls2/users/alevin')
# notebookPath = pathlib.Path.cwd()
maskPath = userPath.joinpath('masks')
jsonPath = userPath.joinpath('local_lib')
propPath = pathlib.Path('/nsls2/data/sst/proposals/2022-2/pass-309180')
zarrPath = propPath.joinpath('zarr_datasets')
exportPath = propPath.joinpath('processed_data')

## Set an RSoXS colormap for later
cm = plt.cm.terrain.copy()
cm.set_bad('purple')

Using PyHyperScattering Version: 0+untagged.470.gf7651d5.dirty


## Load DataArrays 

In [11]:
sorted(zarrPath.glob('raw*w1_*SAXS*'))

[PosixPath('/nsls2/data/sst/proposals/2022-2/pass-309180/zarr_datasets/raw_qxqy_andrew1_PM6-Y6-CF_SAXS.zarr')]

In [12]:
# xr.open_zarr
sorted(zarrPath.glob(f'raw*{segment}*SAXS*'))[0]

NameError: name 'segment' is not defined

In [14]:
conda list

# packages in environment at /nsls2/users/alevin/.conda/envs/nrss:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
accessible-pygments       0.0.4              pyhd8ed1ab_0    conda-forge
aiofiles                  22.1.0             pyhd8ed1ab_0    conda-forge
aiosqlite                 0.18.0             pyhd8ed1ab_0    conda-forge
alabaster                 0.7.13             pyhd8ed1ab_0    conda-forge
alsa-lib                  1.2.3.2              h166bdaf_0    conda-forge
anyio                     3.6.2              pyhd8ed1ab_0    conda-forge
aom                       3.5.0                h27087fc_0    conda-forge
argon2-cffi               21.3.0             pyhd8ed1ab_0    conda-forge
argon2-cffi-bindings      21.2.0           py39hb9d737c_3    conda-forge
asttokens                 2.2.1              pyhd8ed1ab_0    

In [13]:
### How you would load data:
segment = 'w1_'  #identifying segment of .zarr string

raw_saxs = xr.open_zarr(sorted(zarrPath.glob(f'raw*{segment}*SAXS*'))[0]).saxs
raw_waxs = xr.open_zarr(sorted(zarrPath.glob(f'raw*{segment}*WAXS*'))[0]).waxs
integ_saxs = xr.open_zarr(sorted(zarrPath.glob(f'integ*{segment}*SAXS*'))[0]).saxs
integ_waxs = xr.open_zarr(sorted(zarrPath.glob(f'integ*{segment}*WAXS*'))[0]).waxs

ValueError: unrecognized engine zarr must be one of: ['scipy', 'store']

In [6]:
### Load energy lists for facet plots
energies = raw_waxs.energy.data
resonant_energies = energies[16:96]

gif_energies = np.array([])
gif_energies = np.append(energies[0:16:3], energies[-31::2])
gif_energies = np.sort(np.append(gif_energies, resonant_energies))

pol=0

# #### View facet plot to verify data:
# integ_waxs.sel(pol=pol, energy=gif_energies[:-6:6], method='nearest').plot.imshow(xscale='log', xlim=(1e-2, 2e-1),
#                 norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)
# integ_saxs.sel(pol=pol, energy=gif_energies[:-6:6], method='nearest').plot.imshow(xscale='log', xlim=(1e-3, 1e-2),
#                 norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)

# raw_waxs.sel(pol=pol, energy=gif_energies[:-6:6], method='nearest').plot.imshow(x='qx', y='qy',
#                 norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)
# raw_saxs.sel(pol=pol, energy=gif_energies[:-6:6], method='nearest').plot.imshow(x='qx', y='qy',
#                 norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)


## Analysis / related

In [12]:
# ### Misc settings:

# plt.rcParams['savefig.facecolor'] = 'white'

# sample_name = sample_guide[raw_waxs.sample_name]
# detector = detector_guide[raw_waxs.detector]

# lineplot_cmap = plt.cm.viridis(np.linspace(0,0.9, len(energy_list)))

### Intensity exports:

#### 1. Tiff stacks of qxqy detector images & qchi converted images

#### 2. Meaned 2D intensity plots:
    1. Chi-meaned: 360, para/perp wedges, E vs Q 
    2. Q-meaned: Full q, high/low q regions, E vs Chi
    3. Combinations: Chi & Q-meaned


#### 3. Meaned 1D intensity lineplots, selected energies
    1. Chi-meaned: 360, para/perp wedges (20 deg, 30 deg, 40 deg, 50 deg wedges), I vs Q
    2. Q-meaned: Full q, high/low q regions, I vs Q
    3. Combinations: Chi & Q-meaned

In [ ]:
integ_waxs.sel(pol=0, energy=285, method='nearest').plot.imshow(
    norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', xlim=(0.01, 0.1))

### Facet plots

In [ ]:
### Facet plots:

# energy_list =

# integ_saxs.sel(pol=0, energy=energy_list).sel(q=slice(0.0008, 0.0101)).plot.imshow(
#     norm=LogNorm(1e1, 1e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)

# integ_waxs.sel(pol=0, energy=energy_list).sel(q=slice(0.01, 0.1)).plot.imshow(
#     norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)

# raw_waxs.sel(pol=0, energy=np.linspace(280, 298, 20), method='nearest').plot.imshow(
#     norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)



for num in range(10):
    pol = 90
    grid = raw_waxs.sel(pol=pol, energy=energy_list[8*num:8*num+8], method='nearest').plot.imshow(x='qx', y='qy',
                norm=LogNorm(1e1, 5e3), cmap=cm, interpolation='antialiased', col='energy', col_wrap=4)
    grid.set_xlabels('qx [1/Å]')
    grid.set_ylabels('qy [1/Å]') 
   
    sample_name = sample_guide[raw_waxs.sample_name]
    scan_id = raw_waxs.sampleid
    detector = detector_guide[raw_waxs.detector]

    # Create/select folder for scan to save plots:
    tiffsPath = scanPath.joinpath(f'facet_qxqy_frames_{detector}_{int(pol):0>2}deg')
    tiffsPath.mkdir(parents=True, exist_ok=True)
    
    plt.savefig(tiffsPath.joinpath(f'{sample_name}_{detector}_{int(pol):0>2}_f{num}.svg'))

### 1D Intensity Plots

In [ ]:
# 360 chi mean
pol=0
for i, energy in enumerate(energy_list):
    integ_waxs.sel(pol=pol, energy=energy, method='nearest').mean('chi').plot.line(label=round(energy,1), color=lineplot_cmap[i])

plt.title(f'I vs Q, 360 chi avg: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')
plt.xscale('log')
plt.xlabel('Q [1/nm]')
plt.xlim((0.01, 0.1))
plt.yscale('log')
plt.ylabel('Intensity [arb. units]')
plt.ylim((1, 1e4))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Para chi mean
pol=0
for i, energy in enumerate(energy_list):
    integ_waxs.sel(pol=pol, energy=energy, method='nearest').sel(chi=slice(-10, 10)).mean('chi').plot.line(label=round(energy,1), color=lineplot_cmap[i])

plt.title(f'I vs Q, para chi avg: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')
plt.xscale('log')
plt.xlabel('Q [1/nm]')
plt.xlim((0.01, 0.1))
plt.yscale('log')
plt.ylabel('Intensity [arb. units]')
plt.ylim((1, 1e4))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Perp chi mean
pol=0
for i, energy in enumerate(energy_list):
    integ_waxs.sel(pol=pol, energy=energy, method='nearest').sel(chi=slice(80, 100)).mean('chi').plot.line(label=round(energy,1), color=lineplot_cmap[i])

plt.title(f'I vs Q, perp chi avg: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')
plt.xscale('log')
plt.xlabel('Q [1/nm]')
plt.xlim((0.01, 0.1))
plt.yscale('log')
plt.ylabel('Intensity [arb. units]')
plt.ylim((1, 1e4))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Perp chi mean
pol=0
for i, energy in enumerate(energy_list):
    integ_saxs.sel(pol=pol, energy=energy, method='nearest').sel(chi=slice(50, 130)).mean('chi').plot.line(label=round(energy,1), color=lineplot_cmap[i])

plt.title(f'I vs Q, perp chi avg: {sample_guide[integ_saxs.sample_name]}, {detector_guide[integ_saxs.detector]}, pol={pol}')
plt.xscale('log')
plt.xlabel('Q [1/nm]')
plt.xlim((0.002, 0.01))
plt.yscale('log')
plt.ylabel('Intensity [arb. units]')
plt.ylim((1, 2e2))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# integ_waxs.sel(pol=0, energy=energy_list).mean('chi').plot.line(hue='energy', xscale='log', yscale='log', 
#     xlim=(0.01, 0.1), add_legend=False)
# plt.legend(labels=np.round(energy_list, 1))

### Anisotropy Ratios

In [ ]:
integ_saxs.rsoxs.AR()

In [ ]:
pol=0
AR_saxs_p0 = integ_saxs.sel(pol=pol).rsoxs.AR(pol=pol, chi_width=40)
AR_saxs_p0.plot.imshow(x='q', xscale='log', xlim=(0.002, 0.009), xticks=[0.002, 0.01], y='energy', 
                       ylim=(280, 290), cmap=plt.cm.seismic, vmin=-1, vmax=1, interpolation='antialiased', size=6, 
                       aspect=0.6)

plt.xlabel("q [1/Å]")
plt.title(f'AR map, E vs Q: {sample_guide[integ_saxs.sample_name]}, {detector_guide[integ_saxs.detector]}, pol={pol}')

In [ ]:
### Using pcolormesh instead of imshow:

pol=0
AR_waxs_p0 = integ_waxs.sel(pol=pol).rsoxs.AR(pol=pol, chi_width=40)
AR_waxs_p0.plot.pcolormesh(cmap=plt.cm.seismic, size=6, aspect=1, x='q', xscale='log', xlim=(0.01, 0.1),
                          vmin=-1, vmax=1, y='energy', ylim=(282, 290), infer_intervals=True)

plt.xlabel("q [1/Å]")
plt.title(f'AR map, E vs Q: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')
plt.savefig(scanPath.joinpath(f'AR_map_{sample_name}_{detector}_pol{int(pol)}.svg'))

In [ ]:
pol=0
AR_waxs_p0 = integ_waxs.sel(pol=pol).rsoxs.AR(pol=pol, chi_width=40)
AR_waxs_p0.plot.imshow(x='q', xscale='log', xlim=(0.01, 0.09), xticks=[0.01, 0.1], y='energy', ylim=(280, 290),
                       cmap=plt.cm.seismic, vmin=-1, vmax=1, size=6, aspect=0.6)

plt.xlabel("q [1/Å]")
plt.title(f'AR map, E vs Q: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')

In [ ]:
pol=90
AR_saxs_p90 = integ_saxs.sel(pol=pol).rsoxs.AR(pol=pol, chi_width=40)
AR_saxs_p90.plot.imshow(x='q', xscale='log', xlim=(0.002, 0.009), xticks=[0.01, 0.1], y='energy', ylim=(280, 290),
                       cmap=plt.cm.seismic, vmin=-1, vmax=1, interpolation='antialiased', size=6, aspect=0.6)

plt.xlabel("q [1/Å]")
plt.title(f'AR map, E vs Q: {sample_guide[integ_saxs.sample_name]}, {detector_guide[integ_saxs.detector]}, pol={pol}')

In [ ]:
pol=90
AR_waxs_p90 = integ_waxs.sel(pol=pol).rsoxs.AR(pol=pol, chi_width=40)
AR_waxs_p90.plot.imshow(x='q', xscale='log', xlim=(0.01, 0.09), xticks=[0.01, 0.1], y='energy', ylim=(280, 290),
                       cmap=plt.cm.seismic, vmin=-1, vmax=1, interpolation='antialiased', size=6, aspect=0.6)

plt.xlabel("q [1/Å]")
plt.title(f'AR map, E vs Q: {sample_guide[integ_waxs.sample_name]}, {detector_guide[integ_waxs.detector]}, pol={pol}')

In [ ]:
para_waxs_p0 = integ_waxs.sel(chi=slice(-10, 10)).mean('chi').sel(pol=0)
perp_waxs_p0 = integ_waxs.sel(chi=slice(80, 100)).mean('chi').sel(pol=0)
AR_waxs_p90 = (para_waxs_p0-perp_waxs_p0) / (para_waxs_p0+para_waxs_p0)
AR_waxs_p90

In [ ]:
plt.imshow(AR_waxs_p90.data, cmap=plt.cm.seismic, vmin=-1, vmax=1, interpolation='antialiased', origin='lower')
# plt.xscale('log')
# plt.gcf().set(dpi=300)

In [ ]:
AR_waxs_p90.plot(x='q', vmin=-0.8, vmax=0.8, cmap=plt.cm.seismic)
# AR_waxs_p90.plot.imshow(x='q', xlim=(0.01, 0.1), xscale='log',cmap=plt.cm.seismic, vmin=-1, 
#                 vmax=1, interpolation='none')
plt.ylim(275, 300)

In [ ]:
out90 = integ_waxs.sel(pol=90).rsoxs.AR(pol=90)
out90.plot.imshow(x='q', xlim=(0.01, 0.1), xscale='log', y='energy', ylim=(275, 290),cmap=plt.cm.seismic, vmin=-1, 
                vmax=1, interpolation='antialiased')